In [2]:
import sklearn
import numpy as np
import pandas as pd

# data download
from sklearn.datasets import load_iris

iris = load_iris()
iris_data = iris.data
iris_label = iris.target

df = pd.DataFrame(data=iris_data, columns=iris.feature_names)
df['label'] = iris_label
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
from sklearn.utils import shuffle

X, y = shuffle(iris_data, iris_label)
y

array([2, 0, 0, 2, 1, 1, 0, 2, 0, 2, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 2, 0,
       1, 0, 0, 2, 2, 1, 1, 2, 0, 2, 0, 0, 0, 0, 2, 1, 0, 1, 1, 2, 2, 2,
       1, 2, 1, 2, 0, 1, 1, 0, 0, 1, 2, 1, 0, 2, 2, 2, 1, 1, 0, 2, 0, 2,
       2, 1, 2, 0, 2, 2, 2, 2, 0, 1, 1, 1, 0, 1, 2, 0, 2, 1, 2, 0, 0, 1,
       0, 2, 2, 2, 1, 0, 1, 0, 0, 0, 1, 0, 2, 2, 1, 2, 1, 2, 2, 2, 0, 2,
       2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 1, 1, 1, 2, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 2, 1, 1, 1, 1])

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f"X_train: {len(X_train)}\nX_test: {len(X_test)}")

X_train: 120
X_test: 30


In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(f"{name}: {score}")

Nearest Neighbors: 0.8666666666666667
Linear SVM: 0.8666666666666667
RBF SVM: 0.9
Gaussian Process: 0.8666666666666667
Decision Tree: 0.8333333333333334
Random Forest: 0.9
Neural Net: 0.9333333333333333
AdaBoost: 0.9
Naive Bayes: 0.8666666666666667
QDA: 0.9333333333333333


# Decision Tree

의사결정 나무. 스무 고개 같이 각각의 조건에 대해서 Yes No의 분기로 그룹을 형성한다.


설명이 쉽고 확인하기 편하다. 학습 성능이 낮은 경우가 많고 너무 깊게 만들 경우 overfitting이 발생한다.


의사결정 나무를 여러개 학습시켜서 forest를 만들 수 있는데, 이를 random forest라고 한다. 하나의 모델보다는 여러 모델의 투표가 나을 것이라는 가정이다. 최근에 가장 핫한 알고리즘들인 xgboost, light GBM 등의 근간이 된다.

<img src="image/decision_tree.png">

## Entropy

열역학에 등장하는 그 엔트로피가 맞다. 정보이론에서는 엔트로피를 이용해 놀람의 정도, 즉 정보량을 표현한다. 
예를 들어서 해가 동쪽에서 떴다면 아무도 놀라지 않는다. 이는 너무도 자명한 사실이기 때문이다. 반대로 해가 서쪽에서 뜨면 이는 뉴스거리가 될 수 있다. 
아주 이례적인 사건이기 때문이다. 다시 말하면, 해가 서쪽에서 뜨는 것은 정보량이 많은 사건이다.

entropy를 줄이는 방식으로 학습을 한다는 의미는 다시 말해서 예측 가능한 상황이 되도록 학습을 시킨다는 의미이다.
즉 예상치 못한 결과를 최소화하는 모델을 만들겠다는 의미로 받아들일 수 있다.

# Ensemble Methods

## stacking methods

만약에 70점 정도의 점수를 내는 학생이 있다고 생각해보자. 학생은 10문제 중에 3개는 틀릴 것이다. 
그렇다면 70점을 내는 학생들 10명이 모여서 각자 생각하는 정답에 대해서 투표를 한다면? 정답률은 올라갈 것이다. 
Ensemble 방법은 이런 방법을 의미하며 여러 모델을 통해서 더 나은 결과를 도출하는 방법론이다. 

앞서 언급한 decision tree 를 여러 개 쌓아서 random forest를 만들 수 있다.

즉, 여러 모델의 예측 결과를 다시 예측에 이용해서 최종적인 결과를 얻는 방법이다. 

<img src="image/stacking.png">

## boosting methods

오답 문제를 풀듯이 어차피 맞은 문제는 또 맞을 가능성이 높다.
틀린 문제에 가중치를 두어서 학습할 수 있도록 하는 방법론이 boosting이다.

## bagging methods

bootstrap 방법을 이용해 데이터를 복원 추출한다. 이 의미는 데이터의 중복을 허용해서 추출한다는 의미이다. 
이는 데이터의 변산성을 줄이고 모델의 안정성을 높일 수 있는 것으로 알려져 있다. 


# Support Vector Machine

딥러닝 이전 가장 핫했던 분류기의 일종. 현재도 간단한 테스트용으로 자주 쓰인다. 


두 집단의 경계를 찾기 위해 사이 공간을 의미하는 마진을 최대한 넓게한다.


<img src="image/SVM.png">


linear SVM은 선형 경계만 찾을 수 있는 반면, kernel 함수를 이용하면 비선형의 경계 역시 학습이 가능하다.

# Nearest Neighbors

KNN 이라고 불리는 이 알고리즘은 나의 주변에 있는 데이터를 바탕으로 나의 속성을 파악할 수 있다고 가정한다. (끼리끼리 사이언스 같은 개념이랄까?)

판단하고자 하는 하나의 데이터 포인트와 가까운 K 개의 데이터 속성을 통해 판단하고자하는 데이터의 속성을 유추한다.

<img src="image/KNN.png">

일반적으로 K가 커질수록 경계선이 부드러워진다.

<img src="image/KNN_2.png">

# Logistic Regression

일반 선형 회귀에 logit function을 적용한 회귀

로짓 함수는 아래와 같이 생겼다.

<img src="image/logstic_regression.png">

결과 값이 0에서 1 사이로 나오게 되는데, 이를 잘 생각해보면 확률값으로 변화해주는 트릭이라고 생각할 수 있다. 

추후 딥러닝에서도 많이 쓰이는 방법이니 기억해두면 아주 좋다.